# Generation of kml file with DIRAC output

### -- Michel Villanueva

The kml_gen code is called:

In [1]:
from kml_gen import *

We need to import another packages:
* lxml to manage xml format
* copyfile and os to manage files
* zipfile to compress the data and create kmz file

In [2]:
from lxml import etree
from shutil import copyfile
import os, zipfile

The file "sites.csv" contains the coordinates of the sites

In [3]:
data_coordinates = read_data("input/sites.csv", split = ',')
for site in data_coordinates:
    print site

['Site', 'CoordinateX', 'CoordinateY\r']
['Adelaide', '138.57121', '-34.910836\r']
['SIGNET', '14.48767', '46.04217\r']
['AWS_Tokyo', '139.763', '35.784498\r']
['CC1_Krakow', '19.890541', '50.089557\r']
['BINP', '83.11', '54.85\r']
['Beihang', '116.3408333', '39.98\r']
['CINVESTAV', '-99.128603', '19.510869\r']
['DESY', '9.8772', '53.5772\r']
['Hokudai', '141.345669', '43.073511\r']
['Nara-WU', '135.828454', '34.68787\r']
['Niigata', '138.941511', '37.867478\r']
['Osaka-CU', '135.5075357', '34.5927358\r']
['PNNL-CASCADE', '-119.307833', '46.347283\r']
['PNNL-PIC', '-119.307833', '46.347283\r']
['PNNL', '-119.307833', '46.347283\r']
['TIFR', '72.81', '19.91\r']
['TMU', '139.377199', '35.61656\r']
['DIRAC.Tokyo', '139.763', '35.714\r']
['UAS', '-107.381713', '24.826213\r']
['UVic', '-123.31', '48.47\r']
['DIRAC.Yonsei.kr', '126.938575', '37.565807\r']
['CESNET', '14.3886', '50.102\r']
['CNAF', '11.3417', '44.4948\r']
['CYFRONET', '19.90916', '50.068978\r']
['Cosenza', '16.181456', '39.33

The object 'kml_file' contains all the structure of the kml (xml) file

In [4]:
kml_file = etree.Element('kml')
kml_document = etree.Element('Document')
kml_file.append(kml_document)

Directory 'content/' is used to create later the kmz file

In [5]:
if not os.path.exists('./content/'):
    os.mkdir('./content')
    print "'content' directory created."

The info used to generate the map is the output of gbasf2 for the commands 
<pre>$gb2_site_summary</pre>


In [6]:
with open('info/gb2_site_summary.txt', 'r') as f:
    print(f.read())

Site                      Done    Failed   Running   Stalled   Waiting
ARC.SIGNET.si           164981      1868       872         1         1
CLOUD.AWS_Sydney.au       1020      1006        12         0         2
CLOUD.AWS_Tokyo.jp         369       110         1         0         2
CLOUD.AWS_Virginia.u      4002      1470        93         0         5
CLOUD.CC1_Krakow.pl      45411       445       202         0         2
DIRAC.BINP.ru           152344      6303       549         0         2
DIRAC.Beihang.cn         10939      1710       100         0         0
DIRAC.CINVESTAV.mx       47371      1533        94         0         0
DIRAC.DESY.de                2         0         0         0         1
DIRAC.Hokudai.jp          9025        88        40         0         0
DIRAC.Nara-WU.jp         11140       257        20         0         0
DIRAC.Niigata.jp         10418      2122        44         0         2
DIRAC.Osaka-CU.jp        17108       126        35         0         0
DIRAC.

and 
<pre>$gb2_list_se -l</pre>

In [7]:
with open('info/gb2_list_se.txt', 'r') as f:
    print(f.read())

Adelaide-DATA-SE
 Endpoint: coepp-dpm-01.ersa.edu.au:8446/srm/managerv2?SFN=
 Path: /dpm/ersa.edu.au/home/belle/DATA
 Read: Active
 Write: Active
Adelaide-SE
 Endpoint: coepp-dpm-01.ersa.edu.au:8446/srm/managerv2?SFN=
 Path: /dpm/ersa.edu.au/home/belle/belledisk
 Read: Active
 Write: Active
Adelaide-TMP-SE
 Endpoint: coepp-dpm-01.ersa.edu.au:8446/srm/managerv2?SFN=
 Path: /dpm/ersa.edu.au/home/belle/TMP
 Read: Active
 Write: Active
CESNET-DATA-SE
 Endpoint: dpm1.egee.cesnet.cz:8446/srm/managerv2?SFN=
 Path: /dpm/cesnet.cz/home/belle/DATA
 Read: Active
 Write: Active
CESNET-TMP-SE
 Endpoint: dpm1.egee.cesnet.cz:8446/srm/managerv2?SFN=
 Path: /dpm/cesnet.cz/home/belle/TMP
 Read: Active
 Write: Active
CNAF-DATA-SE
 Endpoint: storm-fe-archive.cr.cnaf.infn.it:8444/srm/managerv2?SFN=
 Path: /belle/DATA
 Read: Active
 Write: Active
CNAF-SE
 Endpoint: storm-fe-archive.cr.cnaf.infn.it:8444/srm/managerv2?SFN=
 Path: /
 Read: Active
 Write: Active
CNAF-TMP-SE
 Endpoint: storm-fe-archive.cr.cnaf.i

With  **`read_ce_sites`**  we pull the computing element sites information. Notice that now includes the coordinates.

In [8]:
# Pull the CE sites summary
ce_sites = read_ce_sites('info/gb2_site_summary.txt')
for ce in ce_sites:
    ce.print_info()

NameError: name 'read_ce_sites' is not defined

And the same for the storage element sites

In [ ]:
# Pull the SE sites summary
se_sites = read_se_sites('info/gb2_list_se.txt')
for se in se_sites:
    se.print_info()

In [ ]:
for ce in ce_sites:
    kml_document.append(add_ce_site(ce))
    pie_plot(ce.name, ce.jobs_done, ce.jobs_failed)
for se in se_sites:
    if not 'TMP' in se.name and not 'DATA' in se.name :
        kml_document.append(add_se_site(se))

In [ ]:
kml_writer = open("content/KMLGen.kml","w")
print etree.tostring(kml_file, doctype='<?xml version="1.0" encoding="UTF-8"?>', pretty_print=True)
kml_writer.write(etree.tostring(kml_file, doctype='<?xml version="1.0" encoding="UTF-8"?>', pretty_print=True))
kml_writer.close()

In [ ]:
# Make kmz file
print "Generating KMZ file..."
if not os.path.exists('./output/'):
    os.mkdir('./output')
    print "'output' directory created."
zp = zipfile.ZipFile('output/GridMap.kmz','w')
os.chdir("content")
for file in os.listdir("."):
    zp.write(os.path.join(file))
zp.close()
print "KMZ file generated."

In Google Earth this looks really cool...

In [9]:
from IPython.display import YouTubeVideo
YouTubeVideo("b3toCXxkxew")

And [**this**](http://jaguar.fis.cinvestav.mx/testing/gridmap) is the result when you put the kmz file running on google maps.

In [10]:
from IPython.display import HTML
HTML("<iframe src='http://jaguar.fis.cinvestav.mx/testing/gridmap' width = 600 height = 400></iframe>")